In [2]:

import numpy as np
import itertools
import bisect
import math
import json
import os
class Hypercube:                #超立方體
    '''
    A class to create a hypercube object which stores values on vertices
    and values on the edges between neighboring vertices
    '''    
    #輸入維度、(點鍵值)、(點值)
    def __init__(self, n_vertices, vertex_keys = None, vertex_values = None):   
        self.n_vertices = n_vertices
        self.v_num = 2**self.n_vertices
        self.V = [np.array([])] + all_subsets(n_vertices)   #所有子集包含空集，即所有點
        self.V_value = {str(v) : 0. for v in self.V}         #所有點值  先設為0
        self.E = []
        self.E_value = {}
        self.partial_gradient = {vertex : {} for vertex in range(n_vertices)}   #各個維度的partial gradient
        self.matrix = np.full((self.v_num,self.v_num),np.nan)                   #原始立方體對應到的矩陣
        self.partial_gradient_matrix = np.full((self.v_num,self.v_num),np.nan)  #對特定 feature的partial gradient
        self.vi_matrix = np.full((self.v_num,self.v_num),np.nan)                #v_i對應到的矩陣
        self.res_matrix = np.full((self.v_num,self.v_num),np.nan)               #residual的matrix
        self.sv = 0
 
    def set_vertex_values(self, vertex_values):         #設置點值
        for v in vertex_values:                         #用鍵值來做查找
            self.V_value[v] = vertex_values[v]
            
        # edge values are the differences between neighboring vertex values
        #self._calculate_edges()
        
    def _calculate_edges(self):                 #計算邊值
        
        # calculate the usual gradients: the difference between neighboring edges
        for i, v in enumerate(self.V):
            for _v in self.V[i+1:]:
                if self._vertices_form_a_valid_edge(v, _v):
                    self.E.append((v, _v))
                    self.E_value[str((v, _v))] = self.V_value[str(_v)] - self.V_value[str(v)]
        
        # calculate partial gradients
        for vertex in range(self.n_vertices):
            self.partial_gradient[vertex] = self.E_value.copy()
            for v, _v in self.E:
                is_relevant_edge_for_partial_gradient = (vertex in v and vertex not in _v) or (vertex in _v and vertex not in v)
                if not is_relevant_edge_for_partial_gradient:
                    self.partial_gradient[vertex][str((v, _v))] = 0.
            
    def _vertices_form_a_valid_edge(self, v, _v):       #檢查交集和是否相鄰
        # vertices are neighbors in a hypercube
        # if they differ by exactly one element
        differ_in_size_by_1 = (abs(len(v) - len(_v)) == 1)      #差距是1
    
        the_intersection = np.intersect1d(v, _v)                #兩個集合的交集
        #print(type(v[0]),type(_v[0]),type(the_intersection[0]))
        intersection_is_nonempty = len(the_intersection) > 0 or len(v)==0 or len(_v) == 0   #交集大於0或v,_v是空集
        is_intersection = False                         
        if len(the_intersection)>0:                              
            if len(the_intersection)==len(v) or len(the_intersection)==len(_v):
                is_intersection = True
        else:
            if len(v)==0 and len(_v)==1:
                is_intersection = True
       # print(is_intersection)
        return differ_in_size_by_1 and intersection_is_nonempty and is_intersection
    
    #create matrix for Hypercube
    def trans_to_matrix(self,feature_i):                #超立方體轉換成矩陣
        for i, v in enumerate(self.V):                  #對立方體上所有點
            for j,_v in enumerate(self.V[i+1:]):        
                if self._vertices_form_a_valid_edge(v, _v):     #是否相交
                    self.matrix[i][i+j+1] = self.V_value[str(_v)] - self.V_value[str(v)]    #獲得matrix
                    self.matrix[i+j+1][i] = self.V_value[str(v)] - self.V_value[str(_v)]
        
            
        self.partial_gradient_matrix = self.matrix.copy()       #把原始立方體複製下來，保留i方向
        self.my_list = []
        self.partial_gradient_vector = np.array([])
        self.weight_vector = np.array([])
        self.partial_gradient_norm = 0
        for j, v in enumerate(self.V):
            for k,_v in enumerate(self.V[j+1:]):
                if self._vertices_form_a_valid_edge(v, _v):     #如果是邊
                    is_relevant_edge_for_partial_gradient = (feature_i in v and feature_i not in _v) or (feature_i in _v and feature_i not in v)
                    if not is_relevant_edge_for_partial_gradient:           #如果 特徵i不在v 或_v，把那條邊設成0
                        self.partial_gradient_matrix[j][j+k+1] = 0.
                        self.partial_gradient_matrix[j+k+1][j] = 0.
                    else:
                        self.partial_gradient_norm += (self.partial_gradient_matrix[j][j+k+1])**2       # norm
                        self.partial_gradient_vector = np.append(self.partial_gradient_vector,self.partial_gradient_matrix[j][j+k+1])      #比較向量1
                        subset_len = len(v)
                        weight = 1./(math.comb(self.n_vertices,subset_len)*(self.n_vertices-subset_len))            #Shapley value權重參數
                        self.weight_vector = np.append(self.weight_vector,weight)
                        self.my_list.append((j,j+k+1))          
        #self.vi[feature_i] = self.shapley_residuals_in_matrix()
        #self.vi[i] = self.shapley_residuals_in_matrix
    #minimize the function (gradient_x - partial_gradient_i)^2 最小化l2_norm
    def shapley_residuals_in_matrix(self):
            derivative_i  = np.full((self.v_num,self.v_num),0.)      #計算微分後得到的方程式矩陣 A
            b_i = np.array([0.]*self.v_num)                          #得到Ax = b 的 b向量值
            for j  in range(self.v_num):                            #對矩陣的每個點
                for k in range(self.v_num):
                    if np.isnan(self.partial_gradient_matrix[j][k]):    #不用計算nan
                        continue
                    #elif j == 0 or k ==0:                               #如果是跟原點相鄰
                     #   derivative_i[j][j] += 1.                       #係數+1
                      #  b_i[j] += - self.partial_gradient_matrix[j][k]     #x_j-x_i-partial_gradient
                    else:                                               
                        derivative_i[j][j] += 1.                         
                        derivative_i[j][k] += -1.
                        b_i[j] += - self.partial_gradient_matrix[j][k]
            A = derivative_i[1:,1:]                                    #只要x_i for i!=0
            
            B = b_i[1:]                                                #保留b_i
            res = 0.
            A_inverse = np.linalg.inv(A)                               #算inverse matrix
            vi = np.insert(np.dot(A_inverse,B),0,0.)                    #vi  = b/A #開頭補0
            vi_V =  [np.array([])] + all_subsets(self.n_vertices)
            vi_V_value = {str(v) : 0. for v in vi_V} 
            res_dic = {}
            for k,v in enumerate(vi_V):               
                vi_V_value[str(v)] = vi[k]
            self.vi_vector = np.array([])                              #比較向量#2
            for i, v in enumerate(vi_V):
                for j,_v in enumerate(vi_V[i+1:]):
                    if self._vertices_form_a_valid_edge(v, _v):
                        self.vi_matrix[i][i+j+1] = vi_V_value[str(_v)] - vi_V_value[str(v)]
                        self.vi_matrix[i+j+1][i] = vi_V_value[str(v)] - vi_V_value[str(_v)]
                        self.res_matrix[i][i+j+1] = self.partial_gradient_matrix[i][i+j+1] - self.vi_matrix[i][i+j+1]
                        self.res_matrix[i+j+1][i] = self.partial_gradient_matrix[i+j+1][i] - self.vi_matrix[i+j+1][i]
                        res += (self.res_matrix[i][i+j+1])**2
                        res_dic[str(v)+'->'+str(_v)] = self.res_matrix[i][i+j+1]
                        if (i,i+j+1) in self.my_list:
                            self.vi_vector = np.append(self.vi_vector,self.vi_matrix[i][i+j+1])     #比較向量#2
            #print(self.my_list)
            #print(self.partial_gradient_vector,self.vi_vector)
            vector_A = self.weight_vector*self.partial_gradient_vector
            vector_B = self.weight_vector*self.vi_vector
            cos_sim = self.cos_sim(vector_A,vector_B)
            print('cos_sim = ',cos_sim)
            self.sv += vi[-1]
            print('shapley_value: ',vi[-1],'residual sum: ',res)
            #print(self.sv)
            #print('residuals_sum:',res,'shapley_value: ',vi)
            res =  (res/self.partial_gradient_norm)**0.5
            print('norm: ',res)
            
            return vi_V_value, res_dic, self.partial_gradient_norm
    
    def cos_sim(self,a,b):
        dot_product = np.dot(a, b)
        norm_a = np.linalg.norm(a)
        norm_b = np.linalg.norm(b)
        similarity = dot_product / (norm_a * norm_b)
        return similarity
            
####################     
def save_json(dic,feature_name,folder_path):
    filename = f"dic_{feature_name}.json"
    file_path = os.path.join(folder_path,filename)
    with open(file_path, 'w') as json_file:
        json.dump(dic, json_file)

def save_res_json(dic,feature_name,folder_path):
    filename = f"dic_{feature_name}_res.json"
    file_path = os.path.join(folder_path,filename)
    with open(file_path, 'w') as json_file:
        json.dump(dic, json_file)

        
def all_subsets(n_elts):
    '''
        returns a list of 2^{n_elts} lists
        each a different subset of {1, 2,...,n_elts}
    '''
    res = [np.array(list(itertools.combinations(set(range(n_elts)), i))) for i in range(n_elts)]
    res = {i : res[i] for i in range(n_elts)}
    res[n_elts] = np.array([i for i in range(n_elts)]).reshape(1,-1)
    return [res[i][j] for i in range(1,n_elts+1) for j in range(res[i].shape[0])]

def get_residual(old_cube, new_cube, vertex):   #計算殘差
    '''
    returns: residual dictionary
        
        { edge : ▼_player_v[edge] - ▼v_player[edge] for edge in old_cube }
    '''
    assert set(old_cube.E_value.keys()) == set(new_cube.E_value.keys())     #判斷兩個字典中鍵值的組合是否相同。assert:
    res = {}
    for e in old_cube.E_value.keys():
        res[e] = old_cube.partial_gradient[vertex][e] - new_cube.E_value[e] #對應某特徵的邊相減 即gradient_i_v - gradient_v_i(殘差)
    return res
count = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
#記得添加number of vertex
def residual_norm(old_cube, vertex_values, vertex,num_features):     #old_cube是原本的SHAP得到的立方體
    '''
    old_cube: v, our game
    vertex: player
    vertex_values: v_player, proposed game
    
    assumes that the order of the values in vertex_values align with the order of the values in old_cube.V
    
    returns: || ▼_player_v - ▼v_player ||
    '''
    if count[vertex]==0 :
        count[vertex] += 1
    new_cube = Hypercube(num_features)
    new_cube.set_vertex_values({str(_vertex) : vertex_values[j] for j, _vertex in enumerate(old_cube.V)})   #將數值設定成0.5
    return np.sum([(r)**2 for r in get_residual(old_cube, new_cube, vertex).values()]), get_residual(old_cube, new_cube, vertex).values() #計算所有residual造成的影響加總
#改一下參數
def compute_residuals_v(old_cube,vertex_of_v_i_cube,_v,num_features):            #(instance cube,算出來的v_i cube,這個cube的指定feature)
    new_vertex =  np.append(np.array(0), vertex_of_v_i_cube)
    new_c = Hypercube(num_features)
    coalitions = [np.array([])] + all_subsets(num_features)
    b = {}
    for i, coalition in enumerate(coalitions):
        b[str(coalition)] = new_vertex[i]
    new_c.set_vertex_values(b)
    res = get_residual(old_cube,new_c,_v)
    return(res.values())



In [3]:
'''from itertools import combinations

def generate_all_subsets(num):
    num_set = [i for i in range(num)]
    return [np.array(s) for r in range(num+1) for s in combinations(num_set, r) ]
def all_subsets(n_elts):
        #returns a list of 2^{n_elts} lists
        #each a different subset of {1, 2,...,n_elts}
    res = [np.array(list(itertools.combinations(set(range(n_elts)), i))) for i in range(n_elts)]
    res = {i : res[i] for i in range(n_elts)}
    res[n_elts] = np.array([i for i in range(n_elts)]).reshape(1,-1)
    return [res[i][j] for i in range(1,n_elts+1) for j in range(res[i].shape[0])]
a  = generate_all_subsets(5)

b = [np.array([])]+all_subsets(5)
print(a)
print(b)'''

'from itertools import combinations\n\ndef generate_all_subsets(num):\n    num_set = [i for i in range(num)]\n    return [np.array(s) for r in range(num+1) for s in combinations(num_set, r) ]\ndef all_subsets(n_elts):\n        #returns a list of 2^{n_elts} lists\n        #each a different subset of {1, 2,...,n_elts}\n    res = [np.array(list(itertools.combinations(set(range(n_elts)), i))) for i in range(n_elts)]\n    res = {i : res[i] for i in range(n_elts)}\n    res[n_elts] = np.array([i for i in range(n_elts)]).reshape(1,-1)\n    return [res[i][j] for i in range(1,n_elts+1) for j in range(res[i].shape[0])]\na  = generate_all_subsets(5)\n\nb = [np.array([])]+all_subsets(5)\nprint(a)\nprint(b)'

友達資料處理

In [4]:
import os
import time
import pandas as pd
import numpy as np
import json
import torch
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader
from data_process import read_selected_data, get_y,  split_data, compute_class_weights
from dataset import BertDataset
from model import BertClassifier
from training import train_model
from utils import draw_pics, initial_record
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
import matplotlib.pyplot as plt
total_params = 14
csv_file_path = '/hcds_vol/private/luffy/GANGAN-master/data/processed_data/v014_stage_1.csv'
json_file_path = '/hcds_vol/private/luffy/GANGAN-master/data/controllable_para_v014_14.json'
tool_name = 'ASCVD'
epochs = 50000
lr = 1e-5
batch_size = 1024
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
save_folder_name = 'stage-1-param_'+str(total_params)+'-batch_'+str(batch_size)+'-lr_'+str(lr)
with open(json_file_path, 'r') as f:
    params = json.load(f)
    f.close()
s1_df = pd.read_csv(csv_file_path)
#s1_df.shape
all_key = list(params[tool_name]) 
params_list = []                #取得json檔內的特徵
for key in all_key:
    all_param = params[tool_name][key]
    if(type(all_param) == list):
        for param in all_param:
             params_list.append(param)
    else:
        params_list.append(all_param)
    
# 取得Json檔內包含的特徵
s1_df = s1_df[params_list] 
#print(params)
s1_df.head(10)
feature_df = s1_df.drop(['DFT_CNT'], axis=1)


/hcds_vol/private/luffy/anaconda3/envs/shap_res/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
scaler = StandardScaler()
X_standardized = scaler.fit_transform(feature_df)
nf_df = pd.DataFrame(X_standardized)
nf_df
param_group = [] # [2,2,4,2]
all_key = list(params[tool_name]) # ['EQ', 'PUMP', 'CH', 'VENT', 'y']
all_key.remove('y')

for key in all_key:
    all_value = params[tool_name][key]
    param_group.append(len(all_value))
param_group

[2, 2, 8, 2]

In [6]:
def padding_zero(df, tool_name, total_params, flag, params=params): 
    # 將一維參數matrix擴展為4維
    data_arr = df.to_numpy()
    result = []
    for i in range(len(data_arr)):
        arr_index = 0
        empty_arr = np.zeros((4,total_params)) # chamber數 * 總參數數量
        param_group_cp = param_group.copy()
        for j in range(len(empty_arr)):
            while(param_group_cp[j] > 0):
                empty_arr[j][arr_index] = data_arr[i][arr_index]
                param_group_cp[j] -= 1
                arr_index += 1
        
        if(flag == 1): # bert.py使用
            result.append(empty_arr)
        if(flag == 2): # bert_du.py使用
            result.append(empty_arr.tolist())
    
    if(flag == 1): # bert.py使用
        result = pd.DataFrame({'X': [result[i] for i in range(len(result))]})
    return result

In [7]:
nf_df_4d = padding_zero(nf_df,tool_name,total_params,flag=1)
nf_df_4d_object = nf_df_4d.to_numpy()
nf_df_4d_list = []
for i in range(len(nf_df_4d_object)):
    nf_df_4d_list.append(nf_df_4d_object[i][0])
nf_df_4d_arr = np.array(nf_df_4d_list)

友達資料模型預測及平均

In [8]:
from torch.utils.data import DataLoader, TensorDataset
s1_model_path = '/hcds_vol/private/luffy/GANGAN-master/model/predictor/stage_1_checkpoint.pth'
s1_model =  torch.load(s1_model_path).to(device)
s1_model.eval()
nf_df_4d_tensor = torch.tensor(nf_df_4d_arr,dtype=torch.float)
dataset = TensorDataset(nf_df_4d_tensor)
batch_size = 256
loader = DataLoader(dataset, batch_size=batch_size)
outputs = []
with torch.no_grad():
    for batch_data in loader:
        # 将数据移到指定的设备上（如 CUDA 设备）
        batch_data = batch_data[0].to(device)
        
        # 将数据传递给模型进行推理
        batch_output = s1_model(batch_data)
        probs = (torch.nn.functional.softmax(batch_output, dim=1))
        # 将输出保存起来
        outputs += probs

取得模型平均和對應output

In [9]:

output_arr = np.array([output.cpu().numpy()[0] for output in outputs])
output_df = pd.DataFrame({'Output': output_arr})
new_df = pd.concat([feature_df,output_df],axis=1)
new_df.to_csv('data_with_output.csv',index=False)
new_df



,X_-TACT_TIME_mean,X_-CONVEYOR_SPEED_mean,PUMP_high,PUMP_low,CLN1_over-etching-ratio,CLN1_EPT_time,clean_count,EPT_clean_count_ratio,NH3_TREAT_-RF_FREQ-max,NH3_TREAT_-RF_FREQ-range,NH3_TREAT_-RF_FREQ-mean,NP_3_-MFC_VOL_SIH4-range,VENT_high,VENT_low,Output
0,90,3150,29848.68000,10349.70000,0.001802,9987,3,3329.000,13947,444,13614.0000,0,14870.22222,5360.350000,0.922084
1,90,4200,37183.04000,12891.48000,0.003292,10023,2,5011.500,13963,442,13623.0000,1,16217.42000,8937.611111,0.672468
2,90,4200,40499.96000,12198.94000,0.005368,10059,3,3353.000,13948,418,13649.4286,1,17061.03750,5707.557143,0.576102
3,90,4200,35524.30000,12731.66000,0.008401,9999,3,3333.000,13979,451,13610.0000,1,18121.24667,6610.381818,0.739274
4,90,4200,40188.72000,10520.93333,0.017107,9996,7,1428.000,13967,433,13642.2500,1,16771.88824,9810.666667,0.859217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40971,140,3150,53015.43333,10077.01667,0.001491,12069,8,1508.625,13994,468,13643.0000,1,17537.10000,6312.560000,0.847003
40972,140,3150,53015.43333,12465.30000,0.006489,12021,2,6010.500,13967,441,13636.2500,1,16070.72500,7336.800000,0.829222
40973,140,3150,58385.13333,13051.28571,0.001251,11994,3,3998.000,13975,450,13639.0833,1,16219.02500,6658.808333,0.837974
40974,140,3150,73952.03333,12358.76000,0.006489,12021,3,4007.000,13979,448,13612.4545,1,18019.51429,7409.445455,0.826851


In [10]:
def send_to_model(data):
    if 'Output' in data.columns:
        data = data.drop(columns=['Output'])
    data_standardized = scaler.transform(data)
    data_standardized_df = pd.DataFrame(data_standardized)
    data_4d = padding_zero(data_standardized_df,tool_name,total_params,flag=1)
    data_4d_object =  data_4d.to_numpy()
    data_4d_list = []
    for i in range(len(data_4d_object)):
        data_4d_list.append(data_4d_object[i][0])
    data_4d_arr = np.array(data_4d_list)
    data_4d_tensor = torch.tensor(data_4d_arr,dtype=torch.float)
    my_dataset = TensorDataset(data_4d_tensor)
    batch_size = 256
    my_loader = DataLoader(my_dataset, batch_size=batch_size,num_workers=4)
    data_output = []
    with torch.no_grad():
        for batch_data in my_loader:
        # 将数据移到指定的设备上（如 CUDA 设备）
            batch_data = batch_data[0].to(device)
        
        # 将数据传递给模型进行推理
            batch_output = s1_model(batch_data)
            probs = (torch.nn.functional.softmax(batch_output, dim=1))
        # 将输出保存起来
            data_output += probs
    data_output_arr = np.array([output.cpu().numpy()[0] for output in data_output])
    data_expectation_out = data_output_arr.mean()
    return data_expectation_out

In [11]:
print(params_list)

['X_-TACT_TIME_mean', 'X_-CONVEYOR_SPEED_mean', 'PUMP_high', 'PUMP_low', 'CLN1_over-etching-ratio', 'CLN1_EPT_time', 'clean_count', 'EPT_clean_count_ratio', 'NH3_TREAT_-RF_FREQ-max', 'NH3_TREAT_-RF_FREQ-range', 'NH3_TREAT_-RF_FREQ-mean', 'NP_3_-MFC_VOL_SIH4-range', 'VENT_high', 'VENT_low', 'DFT_CNT']


In [12]:
import numpy as np 
import warnings
import itertools
# 過濾掉FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)
def all_subsets(n_elts):
    '''
        returns a list of 2^{n_elts} lists
        each a different subset of {1, 2,...,n_elts}
    '''
    res = [np.array(list(itertools.combinations(set(range(n_elts)), i))) for i in range(n_elts)]
    res = {i : res[i] for i in range(n_elts)}
    res[n_elts] = np.array([i for i in range(n_elts)]).reshape(1,-1)
    return [res[i][j] for i in range(1,n_elts+1) for j in range(res[i].shape[0])]
para_num = 14
AUO_coalitions = [np.array([])] + all_subsets(para_num) 
coalition_estimated_values = {}
instance_idx = 5
instance = new_df.iloc[instance_idx]
#mean_exp = new_df['Output'].mean()
flag=0
count_n = 1
selected_data = new_df.copy()
selected_data = selected_data[(selected_data['PUMP_low']<20000) & 
                              (selected_data['PUMP_high']>20000) & 
                              (selected_data['VENT_low']<10000) & 
                              (selected_data['VENT_high']>10000) &
                              (selected_data['NH3_TREAT_-RF_FREQ-max']>13800)&
                              (selected_data['NH3_TREAT_-RF_FREQ-mean']<13800)
                                                            ]
print(len(selected_data))
sample_df = selected_data.sample(n=1000,random_state=42)
instance.to_csv('instance_3d.csv',index=True)
sample_df.to_csv('background_dataset_2.csv',index=True)
mean_exp = sample_df['Output'].mean()
print(instance['Output'])
for coalition in AUO_coalitions:
    synth = sample_df.copy()                   #用copy()才不會去更改到原始的dataframe
    if len(coalition)!=0:
        #print(synth.iloc[:,coalition],instance[coalition])
        synth.iloc[:,coalition] = instance.iloc[coalition]
        
        '''if len(coalition)==3 and flag==0:
            print(instance)
            print(synth.head(5))
            flag=1'''
        #if (2 in coalition and 3 not in coalition):
        #    PUMP_high = instance.iloc[2]
        #    synth = synth[synth.iloc[:,3]<PUMP_high]
            #print('good')
            
    #if count_n==100:
        #print('資料集長度:',len(synth))
        #print('feature數:',len(coalition))
        #print('資料: ',synth)
        #print(coalition)
        #count_n = 0

    count_n += 1
    Exp = send_to_model(synth)
    impact = Exp - mean_exp

    coalition_estimated_values[str(coalition)] = impact


31661
0.85921710729599


KeyboardInterrupt: 

In [ ]:
AUOcube = Hypercube(para_num)


In [ ]:

AUOcube.set_vertex_values(coalition_estimated_values)

In [ ]:
print(instance)

In [2]:
p_n = []
instance_folder = f'instance_{instance_idx}_vi_data'
if not os.path.exists(instance_folder):
    os.makedirs(instance_folder)
for i  in range(para_num):
    AUOcube.trans_to_matrix(feature_i=i)
    res = AUOcube.shapley_residuals_in_matrix()
    #print(new_df.columns[i],'residuals_of_feature',i)
    save_json(res[0],feature_name=params_list[i],folder_path=instance_folder)
    save_res_json(res[1],feature_name=params_list[i],folder_path=instance_folder)
    p_n.append(res[2])



NameError: name 'instance_idx' is not defined

In [ ]:
print(p_n)